In [177]:
import numpy as np
import yaml
import pandas as pd
import json
from tqdm import tqdm

In [209]:
SLA = 250
key_name = "rt_ti_u_bound_c90_p95" #rt_ti_u_bound_c90_p95 responseTimes95th
BASE_DIR = '/home/vahid/Dropbox/data/swarm-manager-data/results/nodejs_bookstore/'

In [215]:
#
#"300_120_0.3_10",
#"400_120_0.5_10"
workloads = ["400_120_0.5_10", "500_120_0.65_10", "600_120_0.3_10","700_120_0.5_10"]
cpu_util_rules = sorted(['cpu90>50','cpu90>60','cpu90>70','cpu90>80','cpu90>90',
                         'cpu95>50','cpu95>60','cpu95>70','cpu95>80','cpu95>90',
                         'cpu.mean>50','cpu.mean>60','cpu.mean>70','cpu.mean>80','cpu.mean>90'])
strategies = {
#     'adfc-ti-utilization-2':'/AddDifferentFractionalCPUcores/adfc_utilization_ti_95_'+str(SLA)+'_2.yml',
#     'adfc-rt-utilization-2':'/AddDifferentFractionalCPUcores/adfc_utilization_rt_95_'+str(SLA)+'_2.yml',
#     'mc-adfc-rt-utilization-2':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_rt_95_'+str(SLA)+'_2.yml',
#     'mc-adfc-ti-utilization-2':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_ti_95_'+str(SLA)+'_2.yml',
    
#     'adfc-ti-demand':'/AddDifferentFractionalCPUcores/adfc_demand_ti_95_'+str(SLA)+'.yml',
#     'adfc-rt-demand':'/AddDifferentFractionalCPUcores/adfc_demand_rt_95_'+str(SLA)+'.yml',
#     'mc-adfc-ti-demand':'/AddDifferentFractionalCPUcores/mc_adfc_demand_ti_95_'+str(SLA)+'.yml',
#     'mc-adfc-rt-demand':'/AddDifferentFractionalCPUcores/mc_adfc_demand_rt_95_'+str(SLA)+'.yml',
    
#     'adfc-ti-utilization':'/AddDifferentFractionalCPUcores/adfc_utilization_ti_95_'+str(SLA)+'.yml',
#     'adfc-rt-utilization':'/AddDifferentFractionalCPUcores/adfc_utilization_rt_95_'+str(SLA)+'.yml',
#     'mc-adfc-rt-utilization':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_rt_95_'+str(SLA)+'.yml',
#     'mc-adfc-ti-utilization':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_ti_95_'+str(SLA)+'.yml',

    'afc-ti-0.33': '/AddFractionalCPUcores/afc_0.33_ti_95_' + str(SLA) + '.yml',
    'afc-rt-0.33': '/AddFractionalCPUcores/afc_0.33_rt_95_' + str(SLA) + '.yml',
#     'afc-ti-0.5': '/AddFractionalCPUcores/afc_0.5_ti_95_' + str(SLA) + '.yml',
#     'afc-rt-0.5': '/AddFractionalCPUcores/afc_0.5_rt_95_' + str(SLA) + '.yml',
#     'afc-ti-1': '/AddFractionalCPUcores/afc_1_ti_95_' + str(SLA) + '.yml',
#     'afc-rt-1': '/AddFractionalCPUcores/afc_1_rt_95_' + str(SLA) + '.yml',
}
for rule in cpu_util_rules:
    strategies[rule] = '/CPUUsageIncrease/' + rule.replace('>','_').replace('.','_')+'.yml'

In [216]:
def does_specs_meet_sla(config):
    return config['auth']['responseTimes']['total'][key_name] < SLA and \
            config['books']['responseTimes']['total'][key_name] < SLA

temp_dict = {'workloads':[]}
for strategy, file_path in strategies.items():
    temp_dict[strategy] = []
    for workload in workloads:
        if len(workloads) != len(temp_dict['workloads']): temp_dict['workloads'].append(workload)
        
        meetsSLA = False
        cpu_count = 0
        requiredSteps = 0
        with open(BASE_DIR + workload + file_path) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
        for _step in data['steps']:
            requiredSteps += 1
            cpu_count = _step['specs']['auth']['replicaCount'] * _step['specs']['auth']['CPULimits'] + _step['specs']['books']['replicaCount'] * _step['specs']['books']['CPULimits'] + _step['specs']['gateway']['replicaCount'] * _step['specs']['gateway']['CPULimits']
            if does_specs_meet_sla(_step['info']):
                meetsSLA = True
                break
        
        if not meetsSLA: 
            requiredSteps = 0
            cpu_count = 0
            temp_dict[strategy].append('-')
        else:
            temp_dict[strategy].append((np.round(cpu_count,2), requiredSteps))
#             temp_dict[strategy].append(np.round(cpu_count,2))
#             temp_dict[strategy].append(requiredSteps)

for col, values in temp_dict.items():
    print(col, len(values))
        

workloads 4
afc-ti-0.33 4
afc-rt-0.33 4
cpu.mean>50 4
cpu.mean>60 4
cpu.mean>70 4
cpu.mean>80 4
cpu.mean>90 4
cpu90>50 4
cpu90>60 4
cpu90>70 4
cpu90>80 4
cpu90>90 4
cpu95>50 4
cpu95>60 4
cpu95>70 4
cpu95>80 4
cpu95>90 4


In [217]:
df = pd.DataFrame(temp_dict)
df.head()

,workloads,afc-ti-0.33,afc-rt-0.33,cpu.mean>50,cpu.mean>60,cpu.mean>70,cpu.mean>80,cpu.mean>90,cpu90>50,cpu90>60,cpu90>70,cpu90>80,cpu90>90,cpu95>50,cpu95>60,cpu95>70,cpu95>80,cpu95>90
0,400_120_0.5_10,"(7.2, 6)",-,"(8, 3)","(8, 3)",-,-,-,"(9, 3)","(8, 3)","(8, 3)","(8, 3)",-,"(9, 3)","(8, 3)","(8, 3)","(8, 3)",-
1,500_120_0.65_10,"(9.3, 10)",-,"(9, 4)",-,-,-,-,"(10, 4)","(9, 4)","(10, 6)",-,-,"(11, 4)","(9, 4)","(9, 4)",-,-
2,600_120_0.3_10,"(9.9, 10)","(9.9, 10)","(13, 5)","(10, 4)","(8, 3)","(7, 3)","(7, 4)","(15, 5)","(13, 5)","(11, 4)","(10, 4)","(10, 4)","(15, 5)","(14, 5)","(13, 5)","(10, 4)","(10, 4)"
3,700_120_0.5_10,"(11.4, 11)","(10.5, 10)","(14, 5)",-,-,-,-,"(15, 5)","(14, 5)","(14, 5)",-,-,"(15, 5)","(15, 5)","(14, 5)",-,-


In [203]:
#df.to_csv('/home/vahid/Desktop/cpu-cout.csv', index=False)
df.to_csv('/home/vahid/Desktop/steps-cout.csv', index=False)

In [204]:
df.shape

(4, 24)